# Đồ án cuối kỳ - Phân tích chủ đề văn bản
(Cập nhật 07/01/2021)

Nhóm: 12

Thành viên nhóm:
- Vũ Đăng Hoàng Long - MSSV: 18120203
- Nguyễn Huỳnh Đại Lợi - MSSV: 18120198

---
## Import thư viện

In [4]:
!pip3 install pandas
!pip3 install tqdm
!pip3 install bs4
!pip3 install -U scikit-learn
!pip3 install regex
!pip3 install numpy
import pandas as pd
import regex as re
import time # Dùng để sleep chương trình
from tqdm.notebook import tqdm # Hiện thanh progress cho đẹp :D

# Thư viện để lấy và parse HTML
import requests
from bs4 import BeautifulSoup

# Các thư viện liên quan tới ngôn ngữ và NLP
!pip3 install pyvi ;
!pip3 install gensim ;
!pip3 install underthesea ;
from pyvi import ViTokenizer # thư viện NLP tiếng Việt
from underthesea import word_tokenize
import gensim
import unicodedata

# Thư viện liên quan đến machine learning của Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler,LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import FunctionTransformer

Requirement already up-to-date: scikit-learn in /home/vlozg/.local/lib/python3.8/site-packages (0.24.0)


In [5]:
dir_4 = "src/scraped_data/"
tqdm.pandas()

In [6]:
# Import toàn bộ data (không khuyến cáo nếu máy không nhiều ram)
df = []
for i in range(218):
    df.append(pd.read_csv(dir_4 + f'crawling_{i}.csv'))
    
data_df=pd.concat(df)
data_df.reset_index(drop=True, inplace=True)

In [7]:
data_df.drop(['links', 'title', 'description'],axis=1,inplace=True)

In [8]:
def class_preprocess(df):
    selected_class = ["xã hội", "thế giới", "thể thao", "kinh doanh", "văn hóa", "pháp luật", "sức khỏe", "nhịp sống trẻ",
                      "giáo dục", "thời sự", "nhịp sống số", "tuyển sinh", "du lịch", "phóng sự", "nhà đất", "yêu",
                      "điện ảnh", "tài chính", "âm nhạc", "khoa học", "giải trí", "tv show", "công nghệ", "xe",
                      "thời trang", "smarthome", "đi chơi", "câu chuyện giáo dục", "hồ sơ", "thời sự quốc tế", "ăn gì"]
    processed_df = df[df["class"].isin(selected_class)].copy()
    processed_df.loc[processed_df["class"].isin(["thế giới", "hồ sơ"]), "class"] = "thời sự quốc tế"
    processed_df.loc[processed_df["class"].isin(["xã hội", "thời sự", "phóng sự"]), "class"] = "thời sự trong nước"
    processed_df.loc[processed_df["class"] == "đi chơi", "class"] = "du lịch"
    processed_df.loc[processed_df["class"].isin(["tài chính", "doanh nghiệp"]), "class"] = "kinh doanh"
    processed_df.loc[processed_df["class"] == "nhịp sống số", "class"] = "công nghệ"
    processed_df.loc[processed_df["class"].isin(["âm nhạc", "tv show", "điện ảnh"]), "class"] = "giải trí"
    processed_df.loc[processed_df["class"] == "smarthome", "class"] = "nhà đất"
    processed_df.loc[processed_df["class"].isin(["phòng mạch", "biết để khỏe"]), "class"] = "sức khỏe"
    processed_df.loc[processed_df["class"].isin(["tuyển sinh", "học đường", "câu chuyện giáo dục"]), "class"] = "giáo dục"
    return processed_df

In [9]:
class_preprocess(data_df)["class"].value_counts()

thời sự trong nước    142809
thời sự quốc tế        80010
thể thao               69875
kinh doanh             60805
văn hóa                43199
giáo dục               41822
nhịp sống trẻ          40813
pháp luật              38785
sức khỏe               30409
giải trí               24718
công nghệ              20825
du lịch                13208
nhà đất                10639
yêu                     8922
khoa học                6078
xe                      2823
ăn gì                   2286
thời trang              1605
Name: class, dtype: int64

In [10]:
data_df = class_preprocess(data_df)

In [3]:
!pip3 install pandarallel
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [11]:
%%time
#xoá ký tự thừa
def remove_unnecessary(paragraph):
    temp=re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',paragraph)
    return re.sub(r'\s+', ' ', temp).strip()

data_df['content']=data_df['content'].parallel_apply(remove_unnecessary)

CPU times: user 13.4 s, sys: 5.64 s, total: 19 s
Wall time: 1min 22s


In [13]:
%%time
tqdm.pandas()
#data_df['content']=data_df['content'].parallel_apply(ViTokenizer.tokenize)
data_df['content']=data_df['content'].parallel_apply(lambda x: word_tokenize(x, format="text"))

Process ForkPoolWorker-16:
Process ForkPoolWorker-19:
Process ForkPoolWorker-22:
Process ForkPoolWorker-23:
Process ForkPoolWorker-20:
Process ForkPoolWorker-24:
Process ForkPoolWorker-18:
Process ForkPoolWorker-21:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/li

In [14]:
with open('src/vietnamese-stopwords-dash.txt', encoding='utf-8') as f:
    lines = f.read().splitlines()
    
# Danh sách stopword
stopword = set(lines)

In [15]:
def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)

In [ ]:
%%time
data_df['content']=data_df['content'].parallel_apply(remove_stopwords)

## 6. Tạo pipe tiền xử lý cho đoạn văn bản bất kỳ

## 7. Tách tập train, tập test và tập validation

In [ ]:
y_sr = data_df["class"] # sr là viết tắt của series
X_df = data_df["content"]
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3,stratify=y_sr, random_state=0)

# Feature Engineering đưa dữ liệu dạng văn bản đã được xử lý về dạng vector thuộc tính có dạng số học bằng TF-IDF Vectors ở mức Word level 

In [ ]:
# word level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(train_X_df) # learn vocabulary and idf from training set
train_X_tfidf =  tfidf_vect.transform(train_X_df)
# assume that we don't have test set before
val_X_tfidf =  tfidf_vect.transform(val_X_df)

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(train_X_tfidf)


train_X_tfidf_svd = svd.transform(train_X_tfidf)
val_X_tfidf_svd = svd.transform(val_X_tfidf)

In [ ]:
train_X_tfidf_svd

# Xây dựng mô hình

In [ ]:
encoder =LabelEncoder()
train_y_sr_n = encoder.fit_transform(train_y_sr)
val_y_sr_n = encoder.fit_transform(val_y_sr)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg_model=LogisticRegression()
log_reg_model.fit(train_X_tfidf_svd,train_y_sr_n)


### Độ chính xác tập training

In [ ]:
log_reg_model.score(train_X_tfidf_svd,train_y_sr_n)

### Độ chính xác tập validation

In [ ]:
prediction=log_reg_model.predict(val_X_tfidf_svd)

In [ ]:
1 - (prediction != val_y_sr_n).mean()

# Support Vector Machine (SVM)

In [ ]:
from sklearn.svm import SVC
svm_classifier=SVC()
svm_classifier.fit(train_X_tfidf_svd,train_y_sr_n)

### Độ chính xác tập training

In [ ]:
svm_classifier.score(train_X_tfidf_svd,train_y_sr_n)

### Độ chính xác tập validation

In [ ]:
prediction=svm_classifier.predict(val_X_tfidf_svd)
1 - (prediction != val_y_sr_n).mean()

# MLPClassifier

In [ ]:
mlp_classifier=MLPClassifier(hidden_layer_sizes=(20), activation='tanh',
                                 solver='lbfgs', random_state=0, max_iter=2500)
mlp_classifier.fit(train_X_tfidf_svd,train_y_sr_n)

### Độ chính xác tập training

In [ ]:
mlp_classifier.score(train_X_tfidf_svd,train_y_sr_n)

### Độ chính xác tập validation

In [ ]:
prediction=mlp_classifier.predict(val_X_tfidf_svd)
1 - (prediction != val_y_sr_n).mean()